In [7]:
import cv2
import os
import time
import uuid
import shutil
from sklearn.model_selection import train_test_split

In [34]:
#configuring workspace paths:
TRAIN_IMAGE_PATH = 'C:/Users/91959/Desktop/Personal/Code/SignDine/data/images/train/'
TEST_IMAGE_PATH = '..\\..\\data\\images\\test\\'
TRAIN_ANNOTATION_PATH = '..\\..\\data\\annotations\\train\\result.json'
TEST_ANNOTATION_PATH = '..\\..\\data\\annotations\\test\\result.json'
TRAIN_TFRECORD_PATH = '..\\..\\data\\tf_records\\train\\'
TEST_TFRECORD_PATH = '..\\..\\data\\tf_records\\test\\'

In [9]:
#kebab chicken beef impossible burrito veggie quesadilla hummus guacamole cheese bowl salmon tacos
labels = ['kebab', 'chicken', 'beef', 'impossible', 'burrito', 'veggie', 'quesadilla', 'hummus', 'guacamole', 'cheese', 'bowl', 'salmon', 'tacos']
num_images = 50

In [5]:
#data collection:
cap = cv2.VideoCapture(0)
for label in labels:
   for img_num in range(num_images):
        if img_num == 0:
            ret, frame = cap.read()
            cv2.putText(frame, 'RECORDING FOR LABEL: {}'.format(label), (120, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 4, cv2.LINE_AA)
            cv2.imshow('OpenCV Feed', frame)
        cv2.waitKey(0)
        print(img_num + 1)
        ret, frame = cap.read()
        img_name = os.path.join(IMAGE_PATH, label + '_' + '{}.jpg'.format(str(uuid.uuid1())))
        cv2.imwrite(img_name, frame)
        cv2.imshow('OpenCV Feed', frame)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
1
2
3
4
5
6
7

In [6]:
#creating train and test sets:
split_ratio = 0.2
image_files = [file for file in os.listdir(IMAGE_PATH) if file.endswith(".jpg")]
print("Num_Samples:", len(image_files))
#separate class labels and unique identifiers:
class_labels = [filename.split("_")[0] for filename in image_files]
unique_identifiers = [filename.split("_")[1].split(".")[0] for filename in image_files]
#stratified train-test split:
train_indices, test_indices = train_test_split(range(len(image_files)),
                                               test_size=split_ratio,
                                               stratify=class_labels,
                                               random_state=42)
#move images to respective train and test directories:
for idx in train_indices:
    image_file = image_files[idx]
    shutil.move(os.path.join(IMAGE_PATH, image_file), os.path.join(TRAIN_IMAGE_PATH, image_file))
for idx in test_indices:
    image_file = image_files[idx]
    shutil.move(os.path.join(IMAGE_PATH, image_file), os.path.join(TEST_IMAGE_PATH, image_file))
print("Train and test directories populated successfully.")

Num_Samples: 650
Train and test directories populated successfully.


In [35]:
#train tf-record:
!python -m models.official.vision.data.create_coco_tf_record --logtostderr --image_dir={TRAIN_IMAGE_PATH} --object_annotations_file={TRAIN_ANNOTATION_PATH} --output_file_prefix={TRAIN_TFRECORD_PATH} --num_shards=1

c:\Users\91959\Desktop\Personal\Code\SignDine\venv\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
c:\Users\91959\Desktop\Personal\Code\SignDine\venv\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.11.0 and strictly below 2.14.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bu

In [ ]:
#test tf-record:
!python -m models.official.vision.data.create_coco_tf_record --logtostderr \
--image_dir={TEST_IMAGE_PATH} \
--object_annotations_file={TEST_ANNOTATION_PATH} \
--output_file_prefix={TEST_TFRECORD_PATH} \
--num_shards=1